In [1]:
# # Load Spacy parser
# spacy.load('en')
# parser = English()

# # Function to tokenize text
# def tokenize(text):
#     lda_tokens = []
#     tokens  = parser(text)
#     for token in tokens:
#         if token.orth_.isspace():
#             continue
#         elif token.like_url:
#             lda_tokens.append('URL')
#         elif token.orth_.startswith('@'):
#             lda_tokens.append('SCREEN_NAME')
#         else:
#             lda_tokens.append(token.lower_)
#     return lda_tokens


In [2]:
# # Mallet LDA

# mallet_path = '/Users/ayankarim/mallet-2.0.8/bin/mallet'

# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=3, id2word=id2word, iterations=1000, random_seed=100 )

In [3]:
# # Show topics
# pprint(ldamallet.show_topics(num_words=15, formatted=False))

# # Compute Coherence Score
# coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_ldamallet = coherence_model_ldamallet.get_coherence()
# print('\nCoherence Score: ', coherence_ldamallet)